In [ ]:
from flask import Flask, request, jsonify
import requests
from bs4 import BeautifulSoup

app = Flask(__name__)

def scrape_info_from_website(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  

        soup = BeautifulSoup(response.content, 'html.parser')
        webpage_content = soup.get_text()

        return webpage_content
    except requests.exceptions.RequestException as e:
        return None

def generate_response(webpage_content, user_input):
    try:
        prompt = f"""
Your function today is to answer the user's questions strictly related to the content that is scraped of a webpage. Your reply must be friendly and just like a human. 

**Here's the scraped content of the webpage:**
{webpage_content}

**Here's the user's question:** {user_input}

**Respond with:**

**The answer to the question if it directly relates to the topic.**
**"I don't know the answer" for any question that deviates from the webpage's topic.**
**Limit responses to 5 words or less for off-topic questions.**

**Remember, prioritize staying on topic and avoid going off on tangents.**

**Example:**

User's Question: What is the capital of France?
Answer: I don't know the answer  (<= 5 words) 
"""

        llama_api_url = "http://localhost:11434/api/generate"

        data = {
            "model": "llama3",
            "stream": False,
            "prompt": prompt,
        }

        response = requests.post(llama_api_url, json=data)
        response.raise_for_status()

        response_text = response.json().get("response", "")
        return response_text
    except requests.exceptions.RequestException as e:
        return None

@app.route('/api/answer', methods=['POST'])
def answer_question():
    data = request.json
    url = data.get('url')
    question = data.get('question')

    if not url or not question:
        return jsonify({"error": "Missing URL or question"}), 400

    webpage_content = scrape_info_from_website(url)

    if webpage_content:
        response = generate_response(webpage_content, question)
        return jsonify({"answer": response}), 200 if response else 500
    else:
        return jsonify({"answer": ""}), 200

if __name__ == "__main__":
    app.run(debug=True)
